In [12]:
pip install replicate

DEPRECATION: Loading egg at /Users/jielanzheng/anaconda3/lib/python3.11/site-packages/SQLAlchemy-1.4.23-py3.11-macosx-11.1-arm64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
Note: you may need to restart the kernel to use updated packages.


In [13]:
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

········


In [5]:
import replicate
from tqdm import tqdm

model_name1 = "meta/meta-llama-3-70b-instruct"
model_name2 = "meta/llama-2-70b-chat"

In [49]:
import random

random.seed(123)

## Analysis for QQA
with open("QP/Numeracy600K_comment_test.json", 'r') as file:
    QP_comment = json.loads(file.read())

with open("QP/Numeracy600K_headline_test.json", 'r') as file:
    QP_headline = json.loads(file.read())

QP_comment_sample = random.sample(QP_comment, 600)
QP_headline_sample = random.sample(QP_headline, 600)

In [108]:
def qp_batch(dataset, model_name, result_name):
    answers = {}
    example_prompt = f"""
            Given a comment/headline with masked number, please choose one of the following magnitude for the masked number:
            Option 1: 0, 
            Option 2: 1, 
            Option 3: 2, 
            Option 4: 3, 
            Option 5: 4, 
            Option 6, 5,
            Option 7: 6.
            Example 1: 
            Masked Statement: {dataset[0]['masked']}
            Answer: {dataset[0]['magnitude']}
            Example 2:
            Masked Statement: {dataset[1]['masked']}
            Answer: {dataset[1]['magnitude']}
            Based on the example, answer the following question:
            """

    for i in tqdm(range(len(dataset)-2), desc="Processing Questions"):
        question_prompt = f"""
            Masked Statement: {dataset[i+2]['masked']}
            Please choose one of the following magnitude for the masked number: 
            Option 1: 0, 
            Option 2: 1, 
            Option 3: 2, 
            Option 4: 3, 
            Option 5: 4, 
            Option 6, 5,
            Option 7: 6
            Please keep the answer format as Option num: num
            """
        ans = ''
        for event in replicate.stream(model_name, input={"prompt": example_prompt + question_prompt},):
            ans += str(event)
        answers[i+2] = ans
    
    with open(f'{result_name}.json', 'w', encoding='utf-8') as f:
        json.dump(answers, f, ensure_ascii=False, indent=4)

In [141]:
example_prompt = f"""
            Given a comment/headline with masked number, please choose one of the following magnitude for the masked number:
            Option 1: 0, 
            Option 2: 1, 
            Option 3: 2, 
            Option 4: 3, 
            Option 5: 4, 
            Option 6, 5,
            Option 7: 6.
            Example 1: 
            Masked Statement: {QP_comment_sample[0]['masked']}
            Answer: {QP_comment_sample[0]['magnitude']}
            Example 2:
            Masked Statement: {QP_comment_sample[1]['masked']}
            Answer: {QP_comment_sample[1]['magnitude']}
            Based on the example, answer the following question:
            """
question_prompt = f"""
            Masked Statement: {QP_comment_sample[i+2]['masked']}
            Please choose one of the following magnitude for the masked number: 
            Option 1: 0, 
            Option 2: 1, 
            Option 3: 2, 
            Option 4: 3, 
            Option 5: 4, 
            Option 6, 5,
            Option 7: 6
            Please keep the answer format as Option num: num
            """


            Given a comment/headline with masked number, please choose one of the following magnitude for the masked number:
            Option 1: 0, 
            Option 2: 1, 
            Option 3: 2, 
            Option 4: 3, 
            Option 5: 4, 
            Option 6, 5,
            Option 7: 6.
            Example 1: 
            Masked Statement: GALP ENERGIA SA <GALP.LS> : JEFFERIES RAISES PRICE TARGET TO [Num]0 EUROS FROM 12 EUROS; RATING BUY 
            Answer: 2
            Example 2:
            Masked Statement: BUZZ-USD/MYR hits 4.12%; KLCI -[Num]%
            Answer: 1
            Based on the example, answer the following question:
            
            Masked Statement: BEIJING GEOENVIRON ENGINEERING & TECHNOLOGY <603588.SS> SAYS IT  WINS INDUSTRIAL PARK PROJECT IN HUNAN PROVINCE WITH INVESTMENT  OF [Num] BLN YUAN
            Please choose one of the following magnitude for the masked number: 
            Option 1: 0, 
            Option 2: 1, 
            Opti

In [109]:
qp_batch(QP_comment_sample, model_name1, 'qp_comment')

Processing Questions: 100%|███████████████████| 598/598 [15:38<00:00,  1.57s/it]


In [130]:
qp_batch(QP_headline_sample, model_name1, 'qp_headline')

Processing Questions: 100%|███████████████████| 598/598 [08:48<00:00,  1.13it/s]


In [136]:
def evaluation_qp(results, dataset, all_labels):
    
    confusion_matrix = {label: {'tp': 0, 'fp': 0, 'fn': 0} for label in all_labels}
    
    # Populate the confusion matrix
    for i in range(len(dataset)-2):
        true_label = f'Option {dataset[i+2]["magnitude"] + 1}'
        predicted_label = results[f'{i+2}']

        for label in all_labels:
            if true_label == label:
                if label in predicted_label:
                    confusion_matrix[label]['tp'] += 1
                else:
                    confusion_matrix[label]['fn'] += 1
            elif label in predicted_label:
                confusion_matrix[label]['fp'] += 1

    # Calculate precision, recall, and F1 score for each class
    scores = {}
    for label, counts in confusion_matrix.items():
        precision = counts['tp'] / (counts['tp'] + counts['fp']) if (counts['tp'] + counts['fp']) > 0 else 0
        recall = counts['tp'] / (counts['tp'] + counts['fn']) if (counts['tp'] + counts['fn']) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        scores[label] = {'precision': precision, 'recall': recall, 'f1_score': f1_score}

    # Calculate macro-average F1 score
    macro_f1_score = sum(score['f1_score'] for score in scores.values()) / len(scores)

    return scores, macro_f1_score

In [137]:
## QP
with open("qp_comment.json", 'r') as file:
    QP_Comment = json.loads(file.read())
with open("qp_headline.json", 'r') as file:
    QP_Headline = json.loads(file.read())

all_labels1 = ['Option 1', 'Option 2', 'Option 3', 'Option 4']
all_labels2 = ['Option 1', 'Option 2', 'Option 3', 'Option 4', 'Option 5']
    
print(evaluation_qp(QP_Comment, QP_comment_sample, all_labels1))
print(evaluation_qp(QP_Headline, QP_headline_sample, all_labels2))

({'Option 1': {'precision': 0.5205479452054794, 'recall': 0.2878787878787879, 'f1_score': 0.37073170731707317}, 'Option 2': {'precision': 0.36942675159235666, 'recall': 0.24267782426778242, 'f1_score': 0.29292929292929293}, 'Option 3': {'precision': 0.3228346456692913, 'recall': 0.5061728395061729, 'f1_score': 0.3942307692307692}, 'Option 4': {'precision': 0.05454545454545454, 'recall': 0.05172413793103448, 'f1_score': 0.05309734513274336}}, 0.2777472786524697)
({'Option 1': {'precision': 0.020833333333333332, 'recall': 0.5, 'f1_score': 0.039999999999999994}, 'Option 2': {'precision': 0.4563106796116505, 'recall': 0.1865079365079365, 'f1_score': 0.2647887323943662}, 'Option 3': {'precision': 0.27346938775510204, 'recall': 0.37222222222222223, 'f1_score': 0.31529411764705884}, 'Option 4': {'precision': 0.08163265306122448, 'recall': 0.11764705882352941, 'f1_score': 0.09638554216867469}, 'Option 5': {'precision': 0.0625, 'recall': 0.015625, 'f1_score': 0.025}}, 0.14829367844201996)
